In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import numpy as np
import scipy.io
from sklearn.metrics import hamming_loss
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
import metrics as mt
from sklearn.metrics import label_ranking_average_precision_score
from numpy import linalg as LA
from itertools import combinations
from scipy.linalg import norm
sys.path.append('./function')
import csv
from sklearn.preprocessing import MinMaxScaler


PML-NI用のデータの加工

ラベル数を15に制限する

In [ ]:
def data_for_pmlni(data,candidate_labels,true_labels):
  [num_data,num_fea] = np.shape(data)
  [a,b] = np.shape(candidate_labels)
  if a != num_data:
    candidate_labels = candidate_labels.T
    true_labels = true_labels.T
    num_labels = a
  if num_labels > 100:
    labels_sum = np.zeros((1,num_labels))
    for i in range(0,num_data):
      for j in range(0,num_labels):
        if candidate_labels[i,j] == 1:
          labels_sum[0,j] += 1
    sum_label_rank_ind = np.argsort(labels_sum[0])
    new_cand_labels = np.zeros((num_data,15))
    new_true_labels = np.zeros((num_data,15))
    for i in range(0,15):
      new_cand_labels[:,i] = candidate_labels[:,-sum_label_rank_ind[i]-1]
      new_true_labels[:,i] = true_labels[:,-sum_label_rank_ind[i]-1]
    delete_ind = np.array([],dtype = 'int')
    for i in range(0,num_data):
      a = 0
      for j in range(0,15):
        if new_cand_labels[i,j] == 1:
          a = 1
      if a == 0:
        delete_ind = np.append(delete_ind,i)
    for i in range(0,len(delete_ind)):
      data = np.delete(data,delete_ind[-i-1],0)
      new_cand_labels = np.delete(new_cand_labels,delete_ind[-i-1],0)
      new_true_labels = np.delete(new_true_labels,delete_ind[-i-1],0)
    return(data,new_cand_labels.T,new_true_labels.T)
  else:
    return(data,candidate_labels.T,true_labels.T)

In [ ]:
class PMLNI:
  def __init__(self):
    self.lambd = 10
    self.beta = 0.5
    self.gamma = 0.5
    self.max_iter = 500

  # train_data:M*D (num_data * num_feature)
  # train_target:M*Q (num_label * num_data)
  # true_target:M*Q (num_label * num_data)
  # test_data:N*D (num_data * num_feature)
  def fit(self,train_data,train_target,true_target,test_data):
    lambd = self.lambd
    beta = self.beta
    gamma = self.gamma
    max_iter = self.max_iter

    train_target_T = train_target
    true_target_T = true_target
    train_target = train_target_T.T
    true_target = true_target_T.T
    [num_train,dim] = np.shape(train_data)
    [num_label,a] = np.shape(train_target)
    ##Training
    fea_matrix = np.concatenate([train_data,np.ones((num_train,1))],1)
    U = np.zeros((num_label,dim+1))
    V = np.zeros((num_label,dim+1))
    Y = np.zeros((num_label,dim+1))
    mu = 1e-4
    rho = 1.1

    YX = train_target @ fea_matrix
    XX = fea_matrix.T @ fea_matrix

    for t in range(0,max_iter):
      # update W
      
      # W = (YX+(mu*U)+(mu*V)+Y)/(XX+(lambd+mu)*np.eye(dim+1))
      A = YX+(mu*U)+(mu*V)+Y
      B = XX+(lambd+mu)*np.eye(dim+1)
      W = np.linalg.lstsq(B.T,A.T)[0].T
      #W = np.matmul(A,np.linalg.inv(B))
      #W = dot(A, np.linalg.pinv(B)) 

      # update U V
      Uk = np.copy(U)
      Vk = np.copy(V)
      [M,s,Nhat] = np.linalg.svd(W-V-Y/mu, full_matrices=False)
      svp = len(np.where(s>beta/mu)[0])
      if svp >= 1:
        s = s[0:svp] - beta/mu
      else:
        svp = 1
        s = np.array([0])
      Uhat =M[:,0:svp] @ np.diag(s) @ Nhat[0:svp,:]
      U = np.copy(Uhat)
      
      #L1 NORM
      Vraw = W - U - Y/mu
      Vhat = np.zeros((num_label,dim+1))
      for i in range(0,num_label):
        for j in range(0,dim+1):
          Vhat[i,j] = max(Vraw[i,j]-gamma/mu,0) + min(Vraw[i,j]+gamma/mu,0)
      
      '''# L2,1 NORM
      Vraw = W - U - Y/mu
      Vhat = np.zeros((num_label,dim+1))
      for j in range(0,num_label):
        v = Vraw[j,:]
        vNorm = norm(v)
        if vNorm > gamma/mu:
          Vhat[j,:] = (vNorm-gamma/mu) / vNorm * v'''
      V = np.copy(Vhat)

      # stop criterion
      convg2 = False
      stopCriterion2 = mu*LA.norm(U-Uk,'fro')/LA.norm(W,'fro')
      if stopCriterion2 < 1e-5:
        convg2=True
      convg1 = False
      tmp = W - U - V
      stopCriterion1 = LA.norm(tmp,'fro')/LA.norm(W,'fro')
      if stopCriterion1 < 1e-7:
        convg1 = True
      if convg2:
        mu = min(rho*mu,1e10)
      Y = Y + mu*(U+V-W)

      if convg1 and convg2:
        break
    
    # Computing the size predictor using linear least squares model
    # しきい値の予測器の計算
    Outputs = fea_matrix @ U.T # M*Q
    Left = Outputs
    Right = np.zeros((num_train,1)) # M*1
    for i in range(0,num_train):
      temp = Left[i,:]
      index = np.argsort(temp)
      temp_sorted = np.sort(temp)
      candidate = np.zeros((1,num_label + 1))
      candidate[0,0] = temp[0]-0.1
      for j in range(0,num_label-1):
        candidate[0,j+1] = (temp_sorted[j]+temp_sorted[j+1])/2
      candidate[0,num_label] = temp_sorted[num_label - 1] + 0.1
      miss_class = np.zeros((1,num_label+1))
      for j in range(0,num_label+1):
        temp_notlabels = index[:j]
        temp_labels = index[j:]
        notlabels_true_target = np.ravel(np.where(true_target_T[i,:] != 1))
        labels_true_target = np.ravel(np.where(true_target_T[i,:] == 1))
        false_neg = len(np.array(list(set(temp_notlabels)-set(notlabels_true_target))))
        false_pos = len(np.array(list(set(temp_labels)-set(labels_true_target))))
        miss_class[0,j] = false_neg + false_pos
      temp_index = np.argmin(miss_class)
      Right[i,0] = candidate[0,temp_index]
    Left = np.concatenate([Left,np.ones((num_train,1))],1)

    # for matlab : tempvalue = (Left\Right).T
    tempvalue = np.linalg.lstsq(Left, Right)[0]
    #tempvalue = np.linalg.solve(A,B)
    """num_vars = Left.shape[1]
    rank = np.linalg.matrix_rank(Left)
    print(rank)
    print(num_vars)
    if rank == num_vars:
        tempvalue = np.linalg.lstsq(Left, Right)[0]    # not under-determined
    else:
        for nz in combinations(range(num_vars), rank):    # the variables not set to zero
            try: 
                tempvalue = np.zeros((num_vars, 1))  
                tempvalue[nz, :] = np.asarray(np.linalg.solve(Left[:, nz], Right))
            except np.linalg.LinAlgError:     
                pass"""
    Weights_sizepre = tempvalue[0:num_label]
    Bias_sizepre = tempvalue[num_label:]

    #compute Treshold for each test data
    [num_test, a] = np.shape(test_data)
    Outputs = np.concatenate([test_data,np.ones((num_test,1))],1) @ U.T
    WX = test_data @ U[:,:-1].T
    Threshold = np.concatenate([WX,np.ones((num_test,1))],1) @ np.concatenate([Weights_sizepre,Bias_sizepre])

    # compute result
    Pre_Labels = np.zeros((num_test,num_label))
    for i in range(0,num_test):
      for k in range(0,num_label):
        if Outputs[i,k] >= Threshold[i,0]:
          Pre_Labels[i,k] = 1
        else:
          Pre_Labels[i,k] = 0
    
    self.Outputs = Outputs
    self.Labels = Pre_Labels

  def outputs(self):
    return(self.Outputs)

  def predict(self):
    return(self.Labels)

In [ ]:
import numpy as np
from liblinear.python.liblinear.liblinearutil import *
from PML_NI import PML_NI
#import multilabel_knn.multilabel_knn as mlk
def functions(train_data,train_target,true_target,test_data,test_target,mode):
  if mode == 0: ## PML_NI
    clf = PMLNI()
    clf.fit(train_data,train_target,true_target,test_data)
    print("finish PMLNI_fit")
    creds = clf.outputs()
    print("finish get creds")
    predict_labels = clf.predict()
    print("finish get p_labs")
  elif mode == 1: ## ML_kNN
    model = mlk.multilabel_kNN(k=3, metric = "euclidean") #k: number of neighbors, metric: distance metric {"euclidean", "cosine"}
    model.fit(train_data, train_target) # X :2d feature vectors. Y: label matrix, where Y[i,k] = 1 if i has label k.
    predict_labels = model.predict(test_data).toarray() # X :2d feature vectors. Y: label matrix, where Y[i,k] = 1 if i has label k.
    creds = model.predict(test_data, return_prob = True)[1].toarray()
  elif mode == 2:
    train_target_T = np.array(train_target).T
    test_target_T = np.array(test_target).T
    [num_label,num_train] = np.shape(train_target_T)
    [num_test,num_fea] = np.shape(test_data)
    #%cd /content/drive/My Drive/Colab Notebooks/function/liblinear-2.45
    creds = np.zeros((num_label,num_test))
    predict_labels = np.zeros((num_label,num_test))
    for i in range(0,num_label):
      y_train = train_target_T[i]
      y_test = test_target_T[i]
      m = train(y_train, train_data)
      p_label, p_acc, p_val = predict(y_test, test_data, m)
      predict_labels[i] = np.array(p_label)
      creds[i] = np.ravel(np.array(p_val))
  return(creds,predict_labels)

def get_cvdata(data,candidate_labels,target,crossval_ind,cv_num):
  train_data = []
  train_target = []
  true_target = []
  test_data = []
  test_target = []
  for j in range(0,5):
    if cv_num != j:
      for k in range(0,len(crossval_ind[j])):
        train_data.append(data[crossval_ind[j][k],:])
        train_target.append(candidate_labels[crossval_ind[j][k],:])
        true_target.append(target[crossval_ind[j][k],:])
    elif cv_num == j:
      for k in range(0,len(crossval_ind[j])):
        test_data.append(data[crossval_ind[j][k],:])
        test_target.append(target[crossval_ind[j][k],:])
  train_data = np.array(train_data)
  train_target = np.array(train_target)
  true_target = np.array(true_target)
  test_data = np.array(test_data)
  test_target = np.array(test_target)
  return(train_data,train_target,true_target,test_data,test_target)

def save_result(kind_of_data,creds,predict_labels,mode,percent_of_noise,cv_num):
  func_list = ["PML_NI","ML_kNN","BR"]
  dir_name = "./result/"+str(func_list[mode])+"/"+str(kind_of_data)+"/"+str(percent_of_noise)
  np.savetxt(dir_name+"/creds_"+str(cv_num)+".csv",creds,delimiter=",")
  np.savetxt(dir_name+"/predict_labels_"+str(cv_num)+".csv",predict_labels,fmt="%d",delimiter=",")

In [ ]:
data_list = list([["emotions","enron","CAL500","scene","genbase"],["mirflickr","music_style","music_emotion"],["YeastBP"]])
percent_of_true = 0

In [ ]:
for i in range(1):
  for j in range(1,2):
  #for j in range(1,len(data_list[i])): # kind_of_data
    kind_of_data = data_list[i][j]
    percent_of_noise = 0
    data_fname = kind_of_data + "_data_0_0.csv"
    
    pre_data = np.loadtxt("./datas/"+str(kind_of_data)+"/0/"+data_fname, delimiter=',')
    scaler = MinMaxScaler(feature_range=(0,1))
    data = scaler.fit_transform(pre_data)
    #data = np.loadtxt("./datas/"+str(kind_of_data)+"/0/"+data_fname, delimiter=',')
    target_fname = kind_of_data + "_target_" + str(percent_of_noise) + "_" + str(percent_of_true) +".csv"
    target = np.loadtxt("./datas/"+str(kind_of_data)+"/"+str(percent_of_noise)+"/"+target_fname, delimiter=',')
    with open("./datas/"+str(kind_of_data)+"/cv_inds.csv") as f:
      reader = csv.reader(f)
      r = [row for row in reader]
      crossval_ind = [[int(v) for v in row] for row in r]
      del r
    f.close()
    
    for cv_num in range(1): # cross validation
      if i == 0:
        for l in range(0,1): # percent_of_noise
          percent_of_noise = (l+1)*50
          candidate_labels_fname = kind_of_data + "_candidate_labels_" + str(percent_of_noise)+".csv"
          candidate_labels = np.loadtxt("./datas/"+str(kind_of_data)+"/"+str(percent_of_noise)+"/"+candidate_labels_fname, delimiter=',')
          train_data,train_target,true_target,test_data,test_target = get_cvdata(data,candidate_labels,target,crossval_ind,cv_num)
          for mode in range(1):
            print("start function")
            creds,predict_labels = functions(train_data,train_target,true_target,test_data,test_target,mode)
            print("finish function")
            save_result(kind_of_data,creds,predict_labels,mode,percent_of_noise,cv_num)
            print("finish save result")
      elif i != 0:
        percent_of_noise = 0
        candidate_labels_fname = kind_of_data + "_candidate_labels_" + str(percent_of_noise) + "_" + str(percent_of_true) +".csv"
        candidate_labels = np.loadtxt("./datas/"+str(kind_of_data)+"/"+str(percent_of_noise)+"/"+candidate_labels_fname, delimiter=',')
        train_data,train_target,true_target,test_data,test_target = get_cvdata(data,candidate_labels,target,crossval_ind,cv_num)
        for mode in range(0,1):
            creds,predict_labels = functions(train_data,train_target,true_target,test_data,test_target,mode)
            save_result(kind_of_data,creds,predict_labels,mode,percent_of_noise,cv_num)

start function


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


finish PMLNI_fit
finish get creds
finish get p_labs
finish function
finish save result


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:120: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [ ]:
[aaa,bbb] = np.shape(creds)
indicater = 0
for i in range(aaa):
  for j in range(bbb):
    if creds[i][j] < 0:
      indicater += 1
print(indicater)
print(aaa*bbb)

5638
18073


インスタンス数M、ラベル数Q

creds:M*Q

test_target:M*Q

In [ ]:
i = 0
result[i,0] = hamming_loss(test_target,predict_labels)
result[i,1] = mt.one_error(creds,test_target)
result[i,2] = mt.ranking_loss(creds,test_target)
result[i,3] = mt.coverage(creds,test_target)
result[i,4] = label_ranking_average_precision_score(test_target,creds)

In [ ]:
print(table)

[[0.81734993 0.37554905 0.15097304 0.30856515 0.71607467]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]]


In [ ]:
print(test_target)
print(predict_labels)

[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 1 ... 0 1 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 0. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [ ]:
a = np.array([9,8,7,6,5,4,3,2,1])
print(a[:4]-1)

[8 7 6 5]
